## Libraries and Setup

In [1]:
import os
import pandas as pd
from langchain_openai  import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

from dotenv import load_dotenv, find_dotenv

In [2]:
_ = load_dotenv(find_dotenv())

In [3]:
llm_model = "gpt-4o-mini"

In [4]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [5]:
df = pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


# LLM Chain

In [6]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [7]:
template="What is the best name to describe \
a company that makes {product}?"

In [8]:
prompt = ChatPromptTemplate.from_template(
    template=template
)

In [9]:
chain = prompt | llm

In [10]:
product = "Queen Size Sheet Set"
response = chain.invoke(product)
response.content

"Choosing a name for a company that specializes in queen size sheet sets can be both fun and strategic. It’s important to convey comfort, quality, and a focus on bedding. Here are some suggestions:\n\n1. **Queen's Comfort**\n2. **Majestic Sheets**\n3. **Regal Rest**\n4. **Dreamy Queen Sheets**\n5. **Serene Slumber**\n6. **Lavish Linens**\n7. **Royal Bed Essentials**\n8. **Bedtime Bliss**\n9. **Queen's Haven**\n10. **Supreme Sheet Co.**\n\nRemember to check the availability of the name as a domain if you plan to create a website, and consider how it might fit into your branding and marketing strategy!"

# Simple Sequential Chain

In [11]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [13]:
# prompt template 1
first_template = "What is the best name to describe \
a company that makes {product}?"


first_prompt = ChatPromptTemplate.from_template(
    template=first_template
)

# Chain 1
chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt
)

In [14]:
# prompt template 2
second_template = "Write a 20 words description for the following \
company:{company_name}"

second_prompt = ChatPromptTemplate.from_template(
    template=second_template
)

# chain 2
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt
)

In [15]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True
)

In [16]:
product

'Queen Size Sheet Set'

In [17]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Choosing a name for a company that specializes in queen size sheet sets should evoke comfort, quality, and the specific product focus. Here are some ideas:

1. **Queen Comforts**
2. **Majestic Sheets**
3. **Regal Rest**
4. **Queen's Touch Bedding**
5. **Dreamy Duvets**
6. **Sovereign Sheets**
7. **Royal Slumber**
8. **Serene Sleep Sets**
9. **Cosy Queen Collection**
10. **PureSheet Queen Co.**

Make sure to check for availability of the name and consider how it reflects your brand's identity!
Queen Comforts offers luxurious queen size sheet sets designed for ultimate comfort and quality, transforming your sleep experience into royalty.

> Finished chain.


{'input': 'Queen Size Sheet Set',
 'output': 'Queen Comforts offers luxurious queen size sheet sets designed for ultimate comfort and quality, transforming your sleep experience into royalty.'}

# Sequential Chain

In [18]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [19]:
# prompt template 1: translate to english
first_template = "Translate the following review to english:"
"\n\n{Review}"

first_prompt = ChatPromptTemplate.from_template(
    template=first_template
)

# chain 1: input= Review and output= English_Review
chain_one = LLMChain(
    llm=llm, 
    prompt=first_prompt, 
    output_key="English_Review"
)

In [20]:
second_template = "Can you summarize the following review in 1 sentence:"
"\n\n{English_Review}"

second_prompt = ChatPromptTemplate.from_template(
    template=second_template
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(
    llm=llm, prompt=second_prompt, 
    output_key="summary"
)

In [21]:
# prompt template 3: translate to english
third_template = "What language is the following review:\n\n{Review}"

third_prompt = ChatPromptTemplate.from_template(
    template=third_template
)

# chain 2: input= English_Review and output= summary
chain_three = LLMChain(
    llm=llm, 
    prompt=third_prompt, 
    output_key="language"
)

In [22]:
# prompt template 4: follow up message
fourth_template = "Write a follow up response to the following "\
    "summary in the specified language: "\
    "\n\nSummary: {summary}\n\nLanguage: {language}"

fourth_prompt = ChatPromptTemplate.from_template(
    template=fourth_template
)

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(
    llm=llm, 
    prompt=fourth_prompt,
    output_key="followup_message"
)

In [23]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [24]:
review = df.Review[5]
review

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?"

In [25]:
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'English_Review': 'Of course! Please provide the review that you would like me to translate to English.',
 'summary': "Of course! Please provide the review you'd like me to summarize.",
 'followup_message': 'Bien sûr ! Veuillez fournir le texte de la critique en français, et je me ferai un plaisir de le résumer pour vous.'}

# Router Chain

In [26]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [27]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "history", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [28]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nhistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [29]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [30]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [31]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [33]:
chain = MultiPromptChain(
    router_chain=router_chain, 
    destination_chains=destination_chains, 
    default_chain=default_chain, 
    verbose=True
)

In [34]:
# Physics Question
chain.invoke("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


{'input': 'What is black body radiation?',
 'text': 'Black body radiation refers to the electromagnetic radiation emitted by an idealized object known as a "black body." A black body absorbs all incident radiation, regardless of frequency or angle, and re-emits energy in a characteristic spectrum that depends only on its temperature.\n\nThe key points about black body radiation are:\n\n1. **Temperature Dependence**: The spectrum of radiation emitted by a black body is determined by its temperature. As the temperature increases, the peak wavelength of the emitted radiation shifts to shorter wavelengths (this is described by Wien\'s Law).\n\n2. **Planck\'s Law**: The distribution of emitted radiation across different wavelengths can be described by Planck\'s Law, which quantifies the intensity of radiation emitted at various wavelengths for a given temperature.\n\n3. **Stefan-Boltzmann Law**: The total energy radiated per unit surface area of a black body per unit time (its power output)

In [35]:
# Maths Question
chain.invoke("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


{'input': 'what is 2 + 2',
 'text': 'To solve the problem \\(2 + 2\\), we can break it down into its component parts.\n\n1. **Identify the numbers**: We have two numbers, which are 2 and 2.\n2. **Understand the operation**: The operation here is addition, which combines the two numbers.\n3. **Perform the addition**: When we add 2 and 2 together, we count up from 2 by 2: \n   \\[\n   2 + 2 = 4\n   \\]\n\nPutting it all together, the answer to \\(2 + 2\\) is \\(4\\).'}

In [36]:
# History Question
chain.invoke("For how many years Bristishers ruled sub-continent")



> Entering new MultiPromptChain chain...
history: {'input': 'For how many years did the British rule the Indian subcontinent?'}
> Finished chain.


{'input': 'For how many years did the British rule the Indian subcontinent?',
 'text': 'The British rule in the Indian subcontinent is generally considered to have begun in 1858 and lasted until 1947. This period covers approximately 89 years. However, British involvement in India began much earlier, with the establishment of the British East India Company in the early 17th century. The company gradually extended its control, and by the mid-19th century, it was the dominant power in India.\n\nIf we include the earlier period of British involvement before direct rule, one might consider British influence in India from around 1600 when the East India Company was formed, leading to approximately 347 years of British presence in some form. However, the direct colonial governance through the British Crown began in 1858 after the Indian Rebellion of 1857 and ended with Indian independence and the partition of British India in 1947.'}

In [37]:
# Computer Science Question
chain.invoke("What is the difference between Cache, RAM and ROM")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is the difference between Cache, RAM, and ROM?'}
> Finished chain.


{'input': 'What is the difference between Cache, RAM, and ROM?',
 'text': "Certainly! Let's break down the differences between Cache, RAM, and ROM in a clear and structured way:\n\n### 1. Cache\n\n**Definition:** \nCache is a small-sized type of volatile memory that provides high-speed data access to the CPU. It stores frequently accessed data and instructions to reduce the time it takes to fetch them from the main memory (RAM).\n\n**Characteristics:**\n- **Speed:** Extremely fast, faster than RAM.\n- **Size:** Very small compared to RAM, typically measured in kilobytes (KB) to megabytes (MB).\n- **Volatility:** Volatile memory, meaning it loses its contents when power is turned off.\n- **Levels:** Usually has multiple levels (L1, L2, L3) based on proximity to the CPU core, with L1 being the smallest and fastest.\n\n**Use Cases:** \n- Reduces latency for frequently accessed data and instructions by storing data that the processor is likely to reuse.\n\n### 2. RAM (Random Access Memory)

In [38]:
# Default Chain
chain.invoke("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


{'input': 'Why does every cell in our body contain DNA?',
 'text': 'Every cell in our body contains DNA because DNA carries the genetic information essential for the development, functioning, and reproduction of all living organisms. Here are some reasons why DNA is present in every cell:\n\n1. **Genetic Blueprint**: DNA contains the instructions for building and maintaining an organism. It encodes the information needed to produce proteins, which perform a vast array of functions in the body.\n\n2. **Cell Function and Identity**: Each cell type in the body expresses specific genes that determine its function and identity (for example, muscle cells, nerve cells, and skin cells). Despite the variation in function, the underlying DNA is the same in all cells, ensuring a consistent genetic foundation.\n\n3. **Cell Division and Growth**: When cells divide, they need to replicate their DNA so that each daughter cell receives a complete set of genetic instructions. This ensures that all cell